### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumi猫re (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [3]:
movies.shape

(35479, 3)

In [8]:
movies.isnull().sum()

movie_id     0
movie        0
genre       71
dtype: int64

In [17]:
movies.genre.unique()

array(['Documentary|Short', nan, 'Short|Horror', ...,
       'Animation|Action|Crime|Romance|Thriller',
       'Animation|Comedy|Drama|Family|Fantasy|Sci-Fi',
       'Animation|Short|Family|Musical'], dtype=object)

In [28]:
genres = set()
for genre in movies.genre.unique():
    if str(genre) != 'nan':
        genres = genres|set(list(genre.split('|')))

In [60]:
genres

{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western'}

In [3]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0499549,9,1376753198
2,2,1305591,8,1376742507
3,2,1428538,1,1371307089
4,3,0075314,1,1595468524


In [4]:
reviews.shape

(863866, 4)

In [5]:
reviews.isnull().sum()

user_id      0
movie_id     0
rating       0
timestamp    0
dtype: int64

In [6]:
len(reviews.user_id.unique())

67353

In [7]:
np.mean(reviews.rating), np.max(reviews.rating), np.min(reviews.rating)

(7.315877693994207, 10, 0)

In [23]:
# Use your findings to match each variable to the correct statement in the dictionary
a = 53968
b = 10
c = 7
d = 31245
e = 15
f = 0
g = 4
h = 712337
i = 28

dict_sol1 = {
'The number of movies in the dataset': d,
'The number of ratings in the dataset': h,
'The number of different genres': i,
'The number of unique users in the dataset': a,
'The number missing ratings in the reviews dataset': f,
'The average rating given across all ratings': c,
'The minimum rating given across all ratings': f,
'The maximum rating given across all ratings': b,
}

# Check your solution
t.q1_check(dict_sol1)

Oops!  That doesn't look quite right.  Try again.


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp
* Create month/day/year columns from timestamp as dummies

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [53]:
def create_date_column(df):
    df['date'] = df['movie'].apply(lambda title: title[-5:-1])
    return df

In [54]:
movies = create_date_column(movies)

In [38]:
movies['date'].unique()

array(['1894', '1895', '1896', '1902', '1903', '1908', '1909', '1910',
       '1911', '1912', '1913', '1914', '1916', '1915', '1917', '1918',
       '1919', '1920', '1921', '1922', '1923', '1924', '1925', '1926',
       '1927', '1928', '1929', '1930', '1931', '1932', '1933', '1934',
       '1935', '1936', '1937', '1940', '1938', '1939', '1941', '1942',
       '1944', '1943', '2001', '1945', '1946', '1947', '1948', '1949',
       '1952', '1950', '1951', '1954', '1953', '1955', '1956', '1957',
       '1958', '1959', '1961', '1962', '1960', '1963', '1968', '1964',
       '1965', '1971', '1967', '1966', '1969', '1970', '1972', '1990',
       '1973', '2018', '1974', '1975', '1976', '1978', '1977', '1981',
       '1979', '1982', '1980', '1989', '1986', '1984', '1983', '1987',
       '1985', '1988', '2002', '1992', '1991', '1993', '1994', '1995',
       '1996', '1999', '1997', '1998', '2005', '2000', '2003', '2010',
       '2004', '2008', '1901', '2006', '2007', '2012', '1900', '2021',
      

In [55]:
def dummy_century(df):
    df['century'] = df['date'].apply(lambda date: date[:2] + "00's")
    df = df.merge(pd.get_dummies(df['century']), left_index=True, right_index=True)
    return df

In [56]:
movies = dummy_century(movies)

In [63]:
def dummy_genre(df, genres):
    for genre in genres:
        df[genre] = df['genre'].apply(lambda row: 1 if genre in str(row) else 0)
    return df

In [61]:
genres = genres|{'nan'}

In [64]:
movies = dummy_genre(movies, genres)

In [68]:
movies.head()

0        0
1        0
2        0
3        1
4        0
        ..
37704    1
37705    1
37706    1
37707    1
37708    1
Name: nan, Length: 37709, dtype: int64

In [87]:
def create_date(df):
    df['date'] = pd.to_datetime(reviews['timestamp'], unit='s')#.apply(lambda x: x.date())
    return df

In [88]:
reviews = create_date(reviews)

In [79]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0114508,8,1381006850,2013-10-05
1,2,0499549,9,1376753198,2013-08-17
2,2,1305591,8,1376742507,2013-08-17
3,2,1428538,1,1371307089,2013-06-15
4,3,0075314,1,1595468524,2020-07-23
...,...,...,...,...,...
915365,71257,9893250,10,1613857551,2021-02-20
915366,71257,9898858,3,1585958452,2020-04-04
915367,71258,0172495,10,1587107015,2020-04-17
915368,71258,0414387,10,1587107852,2020-04-17


In [91]:
reviews['date'][0]

Timestamp('2013-10-05 21:00:50')

In [110]:
def dummy_date(df):
    df.loc[:,('year')] = df['date'].dt.year
    df.loc[:,('month')] = df['date'].dt.month
    df.loc[:,('day')] = df['date'].dt.day
    return df


In [111]:
reviews = dummy_date(reviews)

In [100]:
reviews = reviews[['user_id', 'movie_id', 'rating', 'timestamp', 'date']]

In [112]:
reviews

,user_id,movie_id,rating,timestamp,date,year,month,day
0,1,0114508,8,1381006850,2013-10-05 21:00:50,2013,10,5
1,2,0499549,9,1376753198,2013-08-17 15:26:38,2013,8,17
2,2,1305591,8,1376742507,2013-08-17 12:28:27,2013,8,17
3,2,1428538,1,1371307089,2013-06-15 14:38:09,2013,6,15
4,3,0075314,1,1595468524,2020-07-23 01:42:04,2020,7,23
...,...,...,...,...,...,...,...,...
915365,71257,9893250,10,1613857551,2021-02-20 21:45:51,2021,2,20
915366,71257,9898858,3,1585958452,2020-04-04 00:00:52,2020,4,4
915367,71258,0172495,10,1587107015,2020-04-17 07:03:35,2020,4,17
915368,71258,0414387,10,1587107852,2020-04-17 07:17:32,2020,4,17


In [113]:
reviews.to_csv('./reviews_clean.csv')
movies.to_csv('./movies_clean.csv')